Project Part 3: A Deep Learning Model
In this third and final part of the project you will train a deep learning model on your dataset. Note that the best way to do this will likely be to fine-tune an existing deep learning model such as GPT-2, BERT, etc. This is the same as what you will do in Assign 5, except that rather than using the Airline Tweet dataset you will be using your own dataset. Note that it is also possible to train a deep learning model from scratch with either PyTorch or TensorFlow/Keras, but that in the real world it will be more likely that you will want to leverage the cutting edge performance of a pre-trained deep learning model such as those available through huggingface. 

As with Parts 1 and 2, this should be done in a Jupyter notebook and you should add the notebook to the repository where Assign 1 and 2 are. When you are done, you will then get the URL of your project_part3.ipynb notebook in your GitHub repository, and submit that URL here in Canvas. 

Note: Be sure that the output from each cell in you your notebook files is saved to the .ipynb file before your final commit and upload to your GitHub repository.

 

If you are working with a more challenging NLP task such as text summarization or text generation, then Part 3 should be where you fine tune an existing deep learning model for your task (ex: our notebook on text generation w/ GPT-2 fine tuned on Hemingway Links to an external site.). Whereas, Part 2 should be where you used a pre-trained model out of the box without fine tuning it on your dataset (ex: our notebook on text generation w/ a pre-trained GPT-2 model Links to an external site.). 

In [ ]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 4.2 MB/s 
     |████████████████████████████████| 182 kB 80.9 MB/s 
     |████████████████████████████████| 168 kB 75.6 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 168 kB 100.5 MB/s 
     |████████████████████████████████| 166 kB 85.1 MB/s 
     |████████████████████████████████| 166 kB 87.4 MB/s 
     |████████████████████████████████| 162 kB 82.8 MB/s 
     |████████████████████████████████| 162 kB 98.6 MB/s 
     |████████████████████████████████| 158 kB 114.8 MB/s 
     |████████████████████████████████| 157 kB 109.2 MB/s 
     |████████████████████████████████| 157 kB 108.0 MB/s 
     |████████████████████████████████| 157 kB 90.2 MB/s 
     |████████████████████████████████| 157 kB 91.5 MB/s 
     |████████████████████████████████| 157 kB 90.0 MB/s 
     |████████████████████████████████| 157 kB 85.9 MB/s 
     |████████████████████████████████| 157 kB 85.1 MB/s 
     |███████

In [35]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [36]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
# import all of the python modules/packages you'll need here
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel
import matplotlib.pyplot as plt
import random
import numpy as np


In [38]:
MODEL_NAME = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2-medium/snapshots/e852c9080bc759a01663acf5a828d95b261a9903/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2-medium/snapshots/e852c9080bc759a01663acf5a828d95b261a9903/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2-medium/snapshots/e852c9080bc759a01663acf5a828d95b261a9903/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_

Embedding(50259, 1024)

In [ ]:
for name, param in model.named_parameters():
  print(name, param.requires_grad)

wte.weight True
wpe.weight True
h.0.ln_1.weight True
h.0.ln_1.bias True
h.0.attn.c_attn.weight True
h.0.attn.c_attn.bias True
h.0.attn.c_proj.weight True
h.0.attn.c_proj.bias True
h.0.ln_2.weight True
h.0.ln_2.bias True
h.0.mlp.c_fc.weight True
h.0.mlp.c_fc.bias True
h.0.mlp.c_proj.weight True
h.0.mlp.c_proj.bias True
h.1.ln_1.weight True
h.1.ln_1.bias True
h.1.attn.c_attn.weight True
h.1.attn.c_attn.bias True
h.1.attn.c_proj.weight True
h.1.attn.c_proj.bias True
h.1.ln_2.weight True
h.1.ln_2.bias True
h.1.mlp.c_fc.weight True
h.1.mlp.c_fc.bias True
h.1.mlp.c_proj.weight True
h.1.mlp.c_proj.bias True
h.2.ln_1.weight True
h.2.ln_1.bias True
h.2.attn.c_attn.weight True
h.2.attn.c_attn.bias True
h.2.attn.c_proj.weight True
h.2.attn.c_proj.bias True
h.2.ln_2.weight True
h.2.ln_2.bias True
h.2.mlp.c_fc.weight True
h.2.mlp.c_fc.bias True
h.2.mlp.c_proj.weight True
h.2.mlp.c_proj.bias True
h.3.ln_1.weight True
h.3.ln_1.bias True
h.3.attn.c_attn.weight True
h.3.attn.c_attn.bias True
h.3.attn.c

In [39]:
books= pd.read_csv('books_clean_3.csv')
books.head(5)

,title,author,description,language,characterCount,wordCount,descriptionAndTitle
0,Harry Potter and the Half-Blood Prince,J.K. Rowling,The war against Voldemort is not going well; e...,English,1239,205,Harry Potter and the Half-Blood Prince The war...
1,Harry Potter and the Order of the Phoenix,"J.K. Rowling,Mary GrandPré",There is a door at the end of a silent corrido...,English,1185,204,Harry Potter and the Order of the Phoenix Ther...
2,Harry Potter and the Chamber of Secrets,J.K. Rowling,The Dursleys were so mean and hideous that sum...,English,1342,222,Harry Potter and the Chamber of Secrets The Du...
3,Harry Potter and the Prisoner of Azkaban,"J.K. Rowling,Mary GrandPré","For twelve long years, the dread fortress of A...",English,1014,176,Harry Potter and the Prisoner of Azkaban For t...
4,Harry Potter and the Goblet of Fire,"J.K. Rowling,Mary GrandPré",Harry Potter is midway through his training as...,English,1052,184,Harry Potter and the Goblet of Fire Harry Pott...


In [42]:
books = books[books['wordCount'] <= 200]
len(books)

10324

In [43]:
descriptions = books['descriptionAndTitle']
descriptions.head()

3    Harry Potter and the Prisoner of Azkaban For t...
4    Harry Potter and the Goblet of Fire Harry Pott...
5    Harry Potter Boxed Set, Books 1-5 (Harry Potte...
6    Unauthorized Harry Potter Book Seven News: \"H...
7    Harry Potter Collection Six years of magic, ad...
Name: descriptionAndTitle, dtype: object

In [44]:
max_length = max([len(tokenizer.encode(description)) for description in descriptions])
if tokenizer.model_max_length < max_length:
  max_length = tokenizer.model_max_length
max_length

Token indices sequence length is longer than the specified maximum sequence length for this model (2352 > 1024). Running this sequence through the model will result in indexing errors


1024

In [ ]:
tokenizer.vocab_size

50257

In [45]:
class DescriptionDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=512, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [46]:
dataset = DescriptionDataset(descriptions, tokenizer, max_length=max_length)
train_size = int(0.75 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])


In [47]:
train_dataset[0]

(tensor([50257, 13448, 47776,  3454, 32452,   262,  5426,   318, 17157,   319,
         15827,   532,   290,  5295,   284,  2222,  1918,   290,  8166,   284,
          2297, 11930,    13, 36397,   465,  6181,  4097,  1088,   683,    11,
          3454, 32452, 21528,   284,  5587,   379,   262,  2612,   286,   262,
         33128,    13,  2399, 34218,   290, 47687,  1410,   318,   284,  8711,
           262,  2297, 11930,  1751,   532,   290,  4705, 47776,    11, 45524,
          4448,   338,  3367,    11,   318,   284,   307,    11, 11122, 32452,
           262,  5426,   318, 17157,   319, 15827,   532,   290,  5295,   284,
          2222,  1918,   290,  8166,   284,  2297, 11930,    13, 36397,   465,
          6181,  4097,  1088,   683,    11,  3454, 32452, 21528,   284,  5587,
           379,   262,  2612,   286,   262, 33128,    13,  2399, 34218,   290,
         47687,  1410,   318,   284,  8711,   262,  2297, 11930,  1751,   532,
           290,  4705, 47776,    11, 45524,  4448,  

In [48]:
import gc
gc.collect()

30830

In [49]:
wandb.init(
        project="project_part_3",
        config={
            "epochs": 5,
            "batch_size": 20,
            "lr": 1e-3,
            #"dropout": random.uniform(0.01, 0.80),
            })
    
    # Copy your config 
config = wandb.config

In [ ]:
training_args = TrainingArguments(output_dir='/results',
                                  num_train_epochs=config.epochs,
                                  learning_rate=config.lr,
                                  per_device_train_batch_size=config.batch_size,
                                  per_device_eval_batch_size=16,
                                  save_strategy='epoch',
                                  evaluation_strategy='epoch',
                                  logging_strategy='epoch',
                                  remove_unused_columns=True,
                                  no_cuda=False,
                                  #logging_steps=100,
                                  #save_steps=5000,                                   
                                  #warmup_steps=10,
                                  #weight_decay=config.weight_decay, 
                                  #logging_dir='/logs',
                                  load_best_model_at_end=True,
                                  report_to = 'wandb')

In [50]:
training_args = TrainingArguments(output_dir='/results',
                                  num_train_epochs=1,
                                  logging_steps=100,
                                  save_steps=5000,                                   
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  warmup_steps=10,
                                  weight_decay=0.05,  
                                  logging_dir='/logs',
                                  report_to = 'wandb')

PyTorch: setting up devices


In [51]:
trainer = Trainer(model=model, args=training_args,  
                  train_dataset=train_dataset,
                  eval_dataset=val_dataset, 
                  # This custom collate function is necessary 
                  # to built batches of data
                  data_collator=lambda data: 
                  {'input_ids': torch.stack([f[0] for f in data]),       
                  'attention_mask': torch.stack([f[1] for f in data]),
                  'labels': torch.stack([f[0] for f in data])})

TrainOutput(global_step=6057, training_loss=0.5333825958942213, metrics={'train_runtime': 3561.2552, 'train_samples_per_second': 1.701, 'train_steps_per_second': 1.701, 'total_flos': 5625140106756096.0, 'train_loss': 0.5333825958942213, 'epoch': 1.0})

In [53]:
# Start training process!
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9291
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 9291
  Number of trainable parameters = 354825216
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
100,2.834700
200,0.841500
300,0.820200
400,0.748300


Step,Training Loss
100,2.834700
200,0.841500
300,0.820200
400,0.748300
500,0.780700
600,0.776200
700,0.783900
800,0.765200
900,0.788100
1000,0.751200


Saving model checkpoint to /results/checkpoint-5000
Configuration saved in /results/checkpoint-5000/config.json
Model weights saved in /results/checkpoint-5000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9291, training_loss=0.7728254523321934, metrics={'train_runtime': 5158.5235, 'train_samples_per_second': 1.801, 'train_steps_per_second': 1.801, 'total_flos': 8628558152859648.0, 'train_loss': 0.7728254523321934, 'epoch': 1.0})

In [54]:
model.save_pretrained("/models")
tokenizer.save_pretrained("/models")
tokenizer = GPT2Tokenizer.from_pretrained("/models")
model = GPT2LMHeadModel.from_pretrained("/models")

Configuration saved in /models/config.json
Model weights saved in /models/pytorch_model.bin
tokenizer config file saved in /models/tokenizer_config.json
Special tokens file saved in /models/special_tokens_map.json
added tokens file saved in /models/added_tokens.json
loading file vocab.json
loading file merges.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Adding <|startoftext|> to the vocabulary
Adding <|pad|> to the vocabulary
loading configuration file /models/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "predict_s

In [60]:
generated = tokenizer("<|startoftext|> Harry Potter and The Half-Blood Prince", return_tensors="pt").input_ids
sample_outputs = model.generate(generated, no_repeat_ngram_size=2, do_sample=True, top_k=50, max_length=100, top_p=0.95, temperature=0.5, num_return_sequences=3)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  Harry Potter and The Half-Blood Prince Harry and his friends are given a chance to test their mettle against the Dark Lord Voldemort. But will they be able to defeat the powerful wizard and save the world?
1:  Harry Potter and The Half-Blood Prince Harry's first adventure as a wizard begins. In this magical world, the most powerful wizard in the world is a boy named Voldemort. Harry must fight to defeat him, and learn the true meaning of his destiny.
2:  Harry Potter and The Half-Blood Prince Harry and his friends are in trouble when a mysterious, black-haired boy appears in their midst. The boy is Harry's best friend, Ron. But Ron is a wizard, and Harry is not. And Ron's magic is no match for Harry, who is,Harry and their friends were in danger when an mysterious,,black-headed boy appeared in our midst.,Harry is in Trouble whena mysterious black hair boy emerges in your midst, threatening to


In [ ]:
# 🐝 Close your wandb run 
wandb.finish()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 3.5))

axes[0].hist(probs[0], bins=np.logspace(-10, -1, 100), color="C0", edgecolor="C0")
axes[0].set_xscale("log")
axes[0].set_yscale("log")
axes[0].set_title("Probability distribution")
axes[0].set_xlabel("Probability")
axes[0].set_ylabel("Count")
#axes[0].grid(which="major")

axes[1].plot(np.cumsum(np.sort(probs[0])[::-1]), color="black")
axes[1].set_xlim([0, 10000])
axes[1].set_ylim([0.75, 1.01])
axes[1].set_title("Cumulative probability")
axes[1].set_ylabel("Probability")
axes[1].set_xlabel("Token (descending probability)")
#axes[1].grid(which="major")
axes[1].minorticks_on()
#axes[1].grid(which='minor', linewidth='0.5')
top_k_label = 'top-k threshold (k=2000)'
top_p_label = 'nucleus threshold (p=0.95)'
axes[1].vlines(x=2000, ymin=0, ymax=2, color='C0', label=top_k_label)
axes[1].hlines(y=0.95, xmin=0, xmax=10000, color='C1', label=top_p_label, linestyle='--')
axes[1].legend(loc='lower right')
plt.tight_layout()